# AI 로그 분석 시스템 - LLM 체인 테스트

**목적**: LangChain 체인이 다양한 입력에 대해 올바르게 동작하는지 테스트

**요구사항**:
- OPENAI_API_KEY 환경변수 설정
- OpenSearch, Kafka 불필요 (순수 LLM 테스트)

**테스트 항목**:
1. 로그 분석 체인 (log_analysis_chain) - 단일 로그 3개
2. 로그 분석 체인 (log_analysis_chain) - Trace 기반 다중 로그 2개
3. 챗봇 RAG 체인 (chatbot_chain) - 기본 질의 4개
4. 챗봇 RAG 체인 (chatbot_chain) - 대화 히스토리 2개

**총 11개 테스트 케이스**

## 1. 환경 설정

In [14]:
%pip install langchain-openai matplotlib
%pip install python-dotenv langchain langchain-openai langchain-community langchain-text-splitters docx2txt langchain-chroma
%pip install pandas

  Using cached contourpy-1.3.3-cp311-cp311-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ---------------------------------------- 0.0/114.6 kB ? eta -:--:--
     ---------- ---------------------------- 30.7/114.6 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 114.6/114.6 kB 2.2 MB/s eta 0:00:00
  Using cached kiwisolver-1.4.9-cp311-cp311-win_amd64.whl.metadata (6.4 kB)
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/8.1 MB 2.2 MB/s eta 0:00:04
    --------------------------------------- 0.1/8.1 MB 1.7 MB/s eta 0:00:05
   - -------------------------------------- 0.3/8.1 MB 2.0 MB/s eta 0:00:04
   - -------------------------------------- 0.3/8.1 MB 1.9 MB/s eta 0:00:05
   - -------------------------------------- 0.3/8.1 MB 1.6 MB/s eta 0:00:05
   - -------------------------------------- 0.4/8.1 MB 1.4 MB/s eta 0:00:06
   -- ---------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: c:\Users\SSAFY\.pyenv\pyenv-win\versions\3.11.9\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: c:\Users\SSAFY\.pyenv\pyenv-win\versions\3.11.9\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: c:\Users\SSAFY\.pyenv\pyenv-win\versions\3.11.9\python.exe -m pip install --upgrade pip


In [15]:
import sys
import os
from datetime import datetime
from pprint import pprint
import time
import asyncio

# 프로젝트 루트를 Python path에 추가
project_root = os.path.abspath('.')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# .env 파일 로드
from dotenv import load_dotenv
load_dotenv()

print("✅ 환경 설정 완료")
print(f"📍 작업 디렉토리: {os.getcwd()}")
print(f"🔑 OpenAI API Key: {'설정됨' if os.getenv('OPENAI_API_KEY') else '❌ 없음'}")

✅ 환경 설정 완료
📍 작업 디렉토리: c:\SSAFY\third_project\AI\S13P31A306
🔑 OpenAI API Key: 설정됨


In [16]:
# LangChain 체인 및 모델 import
from app.chains.log_analysis_chain import log_analysis_chain
from app.chains.chatbot_chain import chatbot_chain
from langchain_core.messages import HumanMessage, AIMessage

print("✅ LangChain 체인 로드 완료")
print(f"  - log_analysis_chain: {type(log_analysis_chain)}")
print(f"  - chatbot_chain: {type(chatbot_chain)}")

✅ LangChain 체인 로드 완료
  - log_analysis_chain: <class 'langchain_core.runnables.base.RunnableSequence'>
  - chatbot_chain: <class 'langchain_core.runnables.base.RunnableSequence'>


## 2. 로그 분석 체인 테스트 - 단일 로그

다양한 에러 로그에 대해 AI가 올바르게 분석하는지 확인합니다.

In [17]:
# 테스트 케이스 1: 단일 로그 분석 (3개)
single_log_tests = [
    {
        "name": "ERROR - NullPointerException",
        "project_uuid": "550e8400-e29b-41d4-a716-446655440000",
        "data": {
            "service_name": "user-service",
            "level": "ERROR",
            "message": "Failed to process user request",
            "method_name": "getUser",
            "class_name": "UserService",
            "timestamp": "2025-01-02T10:30:00Z",
            "duration": 150,
            "stack_trace": """java.lang.NullPointerException: Cannot invoke 'User.getName()' because 'user' is null
    at com.example.user.UserService.getUser(UserService.java:45)
    at com.example.user.UserController.getUserInfo(UserController.java:23)
    at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)""",
            "additional_context": {"user_id": "12345", "request_ip": "192.168.1.100"}
        }
    },
    {
        "name": "ERROR - Database Connection Timeout",
        "project_uuid": "550e8400-e29b-41d4-a716-446655440000",
        "data": {
            "service_name": "order-service",
            "level": "ERROR",
            "message": "Database connection timeout",
            "method_name": "createOrder",
            "class_name": "OrderRepository",
            "timestamp": "2025-01-02T11:00:00Z",
            "duration": 30000,
            "stack_trace": """org.springframework.dao.DataAccessResourceFailureException: Unable to acquire JDBC Connection
    at org.springframework.jdbc.datasource.DataSourceUtils.getConnection(DataSourceUtils.java:82)
    at com.example.order.OrderRepository.createOrder(OrderRepository.java:67)""",
            "additional_context": {"pool_size": 10, "active_connections": 10, "queue_size": 15}
        }
    },
    {
        "name": "INFO - 정상 처리",
        "project_uuid": "550e8400-e29b-41d4-a716-446655440000",
        "data": {
            "service_name": "payment-service",
            "level": "INFO",
            "message": "Payment processed successfully",
            "method_name": "processPayment",
            "class_name": "PaymentService",
            "timestamp": "2025-01-02T12:00:00Z",
            "duration": 250,
            "stack_trace": "N/A",
            "additional_context": {"amount": 50000, "method": "card", "transaction_id": "TXN-123456"}
        }
    }
]

print(f"📋 단일 로그 테스트: {len(single_log_tests)}개 케이스 준비 완료")

📋 단일 로그 테스트: 3개 케이스 준비 완료


In [18]:
# 단일 로그 분석 실행
single_log_results = []

for i, test_case in enumerate(single_log_tests, 1):
    print(f"\n{'='*70}")
    print(f"🧪 테스트 {i}/{len(single_log_tests)}: {test_case['name']}")
    print(f"📦 Project UUID: {test_case['project_uuid']}")
    print(f"{'='*70}")
    
    start_time = time.time()
    
    try:
        # LLM 체인 실행 (비동기)
        result = await log_analysis_chain.ainvoke(test_case['data'])
        
        elapsed_time = time.time() - start_time
        
        print(f"\n✅ 분석 완료 ({elapsed_time:.2f}초)")
        print(f"\n📊 분석 결과:")
        print(f"  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
        print(f"  📝 Summary: {result.summary}")
        print(f"  🔍 Error Cause: {result.error_cause or 'N/A'}")
        print(f"  💡 Solution: {result.solution or 'N/A'}")
        print(f"  🏷️  Tags: {', '.join(result.tags)}")
        print(f"  📈 Analysis Type: {result.analysis_type}")
        
        single_log_results.append({
            "test_name": test_case['name'],
            "project_uuid": test_case['project_uuid'],
            "status": "✅ 성공",
            "time": f"{elapsed_time:.2f}s",
            "summary": result.summary[:80] + "..." if len(result.summary) > 80 else result.summary,
            "tags_count": len(result.tags),
            "has_error_cause": "Yes" if result.error_cause else "No",
            "has_solution": "Yes" if result.solution else "No"
        })
        
    except Exception as e:
        elapsed_time = time.time() - start_time
        print(f"\n❌ 실패: {e}")
        single_log_results.append({
            "test_name": test_case['name'],
            "project_uuid": test_case['project_uuid'],
            "status": "❌ 실패",
            "time": f"{elapsed_time:.2f}s",
            "error": str(e)[:100]
        })


🧪 테스트 1/3: ERROR - NullPointerException
📦 Project UUID: 550e8400-e29b-41d4-a716-446655440000

✅ 분석 완료 (2.71초)

📊 분석 결과:
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  📝 Summary: NullPointerException in UserService.getUser() method
  🔍 Error Cause: User object was null when accessing getName() method
  💡 Solution: Add null check before accessing user object properties
  🏷️  Tags: NullPointerException, UserService, critical
  📈 Analysis Type: AnalysisType.SINGLE

🧪 테스트 2/3: ERROR - Database Connection Timeout
📦 Project UUID: 550e8400-e29b-41d4-a716-446655440000

✅ 분석 완료 (2.82초)

📊 분석 결과:
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  📝 Summary: Database connection timeout in OrderRepository.createOrder() method
  🔍 Error Cause: All database connections are in use, leading to a timeout when trying to acquire a new connection.
  💡 Solution: Increase the database connection pool size or optimize the usage of existing connections.
  🏷️  Tags: DatabaseConnection, Timeout, OrderService, criti

## 3. 로그 분석 체인 테스트 - Trace 기반 다중 로그

같은 trace_id로 묶인 여러 로그를 함께 분석합니다.

In [19]:
# 테스트 케이스 2: Trace 기반 다중 로그 분석 (2개)
trace_log_tests = [
    {
        "name": "Trace - 5개 관련 로그 (API → Service → Error)",
        "project_uuid": "550e8400-e29b-41d4-a716-446655440000",
        "data": {
            "trace_id": "abc123def456",
            "total_logs": 5,
            "center_log_id": 12345,
            "service_name": "user-service",
            "logs_context": """--- Log 1/5 ---
Timestamp: 2025-01-02T10:30:00.000Z
Level: INFO
Service: api-gateway
Class: GatewayController
Method: handleRequest
Message: Incoming request: GET /users/12345

--- Log 2/5 ---
Timestamp: 2025-01-02T10:30:00.100Z
Level: INFO
Service: user-service
Class: UserController
Method: getUser
Message: Fetching user with ID: 12345

--- Log 3/5 ---
Timestamp: 2025-01-02T10:30:00.150Z
Level: ERROR
Service: user-service
Class: UserService
Method: getUserById
Message: NullPointerException occurred
Stack Trace:
java.lang.NullPointerException: Cannot invoke 'User.getName()' because 'user' is null
    at com.example.user.UserService.getUserById(UserService.java:45)

--- Log 4/5 ---
Timestamp: 2025-01-02T10:30:00.155Z
Level: ERROR
Service: user-service
Class: UserController
Method: getUser
Message: Failed to retrieve user

--- Log 5/5 ---
Timestamp: 2025-01-02T10:30:00.160Z
Level: ERROR
Service: api-gateway
Class: GatewayController
Method: handleRequest
Message: Request failed with status 500"""
        }
    },
    {
        "name": "Trace - 12개 로그 (Map-Reduce 트리거)",
        "project_uuid": "550e8400-e29b-41d4-a716-446655440000",
        "data": {
            "trace_id": "xyz789ghi012",
            "total_logs": 12,
            "center_log_id": 67890,
            "service_name": "order-service",
            "logs_context": """--- Log 1/12 ---
Timestamp: 2025-01-02T11:00:00.000Z
Level: INFO
Message: [api-gateway] Order creation request received

--- Log 2/12 ---
Timestamp: 2025-01-02T11:00:00.050Z
Level: INFO
Message: [order-service] Validating order items

--- Log 3/12 ---
Timestamp: 2025-01-02T11:00:00.100Z
Level: INFO
Message: [inventory-service] Checking stock availability

--- Log 4/12 ---
Timestamp: 2025-01-02T11:00:00.150Z
Level: WARN
Message: [inventory-service] Low stock for item A123

--- Log 5/12 ---
Timestamp: 2025-01-02T11:00:00.200Z
Level: INFO
Message: [order-service] Calculating order total

--- Log 6/12 ---
Timestamp: 2025-01-02T11:00:00.250Z
Level: INFO
Message: [payment-service] Processing payment

--- Log 7/12 ---
Timestamp: 2025-01-02T11:00:00.300Z
Level: INFO
Message: [payment-service] Payment gateway contacted

--- Log 8/12 ---
Timestamp: 2025-01-02T11:00:00.350Z
Level: ERROR
Message: [payment-service] Payment gateway timeout
Stack Trace: SocketTimeoutException: Read timed out

--- Log 9/12 ---
Timestamp: 2025-01-02T11:00:00.355Z
Level: ERROR
Message: [order-service] Payment processing failed

--- Log 10/12 ---
Timestamp: 2025-01-02T11:00:00.360Z
Level: INFO
Message: [order-service] Rolling back order

--- Log 11/12 ---
Timestamp: 2025-01-02T11:00:00.410Z
Level: INFO
Message: [inventory-service] Stock reservation released

--- Log 12/12 ---
Timestamp: 2025-01-02T11:00:00.450Z
Level: ERROR
Message: [api-gateway] Order creation failed"""
        }
    }
]

print(f"📋 Trace 로그 테스트: {len(trace_log_tests)}개 케이스 준비 완료")

📋 Trace 로그 테스트: 2개 케이스 준비 완료


In [20]:
# Trace 기반 로그 분석 실행
trace_log_results = []

for i, test_case in enumerate(trace_log_tests, 1):
    print(f"\n{'='*70}")
    print(f"🧪 테스트 {i}/{len(trace_log_tests)}: {test_case['name']}")
    print(f"📦 Project UUID: {test_case['project_uuid']}")
    print(f"🔗 Trace ID: {test_case['data']['trace_id']}")
    print(f"{'='*70}")
    
    start_time = time.time()
    
    try:
        # LLM 체인 실행 (비동기)
        result = await log_analysis_chain.ainvoke(test_case['data'])
        
        elapsed_time = time.time() - start_time
        
        print(f"\n✅ Trace 분석 완료 ({elapsed_time:.2f}초)")
        print(f"\n📊 분석 결과:")
        print(f"  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
        print(f"  📝 Summary: {result.summary}")
        print(f"  🔍 Error Cause: {result.error_cause or 'N/A'}")
        print(f"  💡 Solution: {result.solution or 'N/A'}")
        print(f"  🏷️  Tags: {', '.join(result.tags)}")
        print(f"  📈 Analysis Type: {result.analysis_type}")
        
        trace_log_results.append({
            "test_name": test_case['name'],
            "project_uuid": test_case['project_uuid'],
            "trace_id": test_case['data']['trace_id'],
            "total_logs": test_case['data']['total_logs'],
            "status": "✅ 성공",
            "time": f"{elapsed_time:.2f}s",
            "summary": result.summary[:80] + "..." if len(result.summary) > 80 else result.summary,
            "tags_count": len(result.tags)
        })
        
    except Exception as e:
        elapsed_time = time.time() - start_time
        print(f"\n❌ 실패: {e}")
        trace_log_results.append({
            "test_name": test_case['name'],
            "project_uuid": test_case['project_uuid'],
            "status": "❌ 실패",
            "time": f"{elapsed_time:.2f}s",
            "error": str(e)[:100]
        })


🧪 테스트 1/2: Trace - 5개 관련 로그 (API → Service → Error)
📦 Project UUID: 550e8400-e29b-41d4-a716-446655440000
🔗 Trace ID: abc123def456

✅ Trace 분석 완료 (2.78초)

📊 분석 결과:
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  📝 Summary: NullPointerException in UserService.getUserById() method caused the request to fail.
  🔍 Error Cause: User object was null when accessing getName() method
  💡 Solution: Add null check before accessing user object properties in UserService.getUserById() method.
  🏷️  Tags: NullPointerException, UserService, UserController, api-gateway, critical
  📈 Analysis Type: AnalysisType.TRACE_BASED

🧪 테스트 2/2: Trace - 12개 로그 (Map-Reduce 트리거)
📦 Project UUID: 550e8400-e29b-41d4-a716-446655440000
🔗 Trace ID: xyz789ghi012

✅ Trace 분석 완료 (3.12초)

📊 분석 결과:
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  📝 Summary: Order creation process failed due to a payment gateway timeout, leading to a rollback of the order and subsequent failure notification to the API gateway.
  🔍 Error Cause: Pay

## 4. 챗봇 체인 테스트 - 기본 질의

다양한 질문에 대해 챗봇이 올바르게 응답하는지 확인합니다.

In [21]:
# 테스트 케이스 3: 챗봇 기본 질의 (4개)
chatbot_basic_tests = [
    {
        "name": "한국어 - 에러 조회",
        "project_uuid": "550e8400-e29b-41d4-a716-446655440000",
        "context_logs": """Log 1:
- ID: 12345
- Timestamp: 2025-01-02T10:30:00Z
- Service: user-service
- Level: ERROR
- Message: NullPointerException in UserService.getUser()
- Method: UserService.getUser
- Stack Trace: java.lang.NullPointerException: Cannot invoke 'User.getName()' because 'user' is null

Log 2:
- ID: 12346
- Timestamp: 2025-01-02T10:35:00Z
- Service: user-service
- Level: ERROR
- Message: NullPointerException in UserService.updateUser()
- Method: UserService.updateUser
- Stack Trace: java.lang.NullPointerException: Cannot invoke 'User.setName()' because 'user' is null

Log 3:
- ID: 12347
- Timestamp: 2025-01-02T10:40:00Z
- Service: order-service
- Level: ERROR
- Message: Database connection timeout
- Stack Trace: org.springframework.dao.DataAccessResourceFailureException""",
        "question": "최근에 어떤 에러가 발생했어?"
    },
    {
        "name": "영어 - 에러 조회",
        "project_uuid": "550e8400-e29b-41d4-a716-446655440000",
        "context_logs": """Log 1:
- ID: 12347
- Timestamp: 2025-01-02T11:00:00Z
- Service: order-service
- Level: ERROR
- Message: Database connection timeout
- Method: OrderRepository.createOrder
- Stack Trace: org.springframework.dao.DataAccessResourceFailureException

Log 2:
- ID: 12348
- Timestamp: 2025-01-02T11:05:00Z
- Service: payment-service
- Level: ERROR
- Message: Payment gateway timeout
- Stack Trace: SocketTimeoutException: Read timed out""",
        "question": "What errors occurred recently?"
    },
    {
        "name": "한국어 - 해결 방법 질문",
        "project_uuid": "550e8400-e29b-41d4-a716-446655440000",
        "context_logs": """Log 1:
- ID: 12345
- Timestamp: 2025-01-02T10:30:00Z
- Service: user-service
- Level: ERROR
- Message: NullPointerException in UserService.getUser()
- Stack Trace: java.lang.NullPointerException: Cannot invoke 'User.getName()' because 'user' is null
    at com.example.user.UserService.getUser(UserService.java:45)""",
        "question": "NullPointerException을 어떻게 해결하면 돼?"
    },
    {
        "name": "컨텍스트 없음 - 일반 질문",
        "project_uuid": "550e8400-e29b-41d4-a716-446655440000",
        "context_logs": "No relevant logs found.",
        "question": "최근 에러 알려줘"
    }
]

print(f"📋 챗봇 기본 질의 테스트: {len(chatbot_basic_tests)}개 케이스 준비 완료")

📋 챗봇 기본 질의 테스트: 4개 케이스 준비 완료


In [22]:
# 챗봇 기본 질의 테스트 실행
chatbot_basic_results = []

for i, test_case in enumerate(chatbot_basic_tests, 1):
    print(f"\n{'='*70}")
    print(f"💬 테스트 {i}/{len(chatbot_basic_tests)}: {test_case['name']}")
    print(f"📦 Project UUID: {test_case['project_uuid']}")
    print(f"❓ 질문: {test_case['question']}")
    print(f"{'='*70}")
    
    start_time = time.time()
    
    try:
        # 챗봇 체인 실행 (비동기)
        response = await chatbot_chain.ainvoke({
            "context_logs": test_case['context_logs'],
            "question": test_case['question'],
            "chat_history": []  # 히스토리 없음
        })
        
        elapsed_time = time.time() - start_time
        answer = response.content
        
        print(f"\n✅ 응답 생성 완료 ({elapsed_time:.2f}초)")
        print(f"\n💡 답변:")
        print(f"  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
        print(f"  {answer}")
        print(f"  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
        
        chatbot_basic_results.append({
            "test_name": test_case['name'],
            "project_uuid": test_case['project_uuid'],
            "question": test_case['question'][:50],
            "status": "✅ 성공",
            "time": f"{elapsed_time:.2f}s",
            "answer_length": len(answer),
            "answer_preview": answer[:100] + "..." if len(answer) > 100 else answer
        })
        
    except Exception as e:
        elapsed_time = time.time() - start_time
        print(f"\n❌ 실패: {e}")
        chatbot_basic_results.append({
            "test_name": test_case['name'],
            "project_uuid": test_case['project_uuid'],
            "question": test_case['question'][:50],
            "status": "❌ 실패",
            "time": f"{elapsed_time:.2f}s",
            "error": str(e)[:100]
        })


💬 테스트 1/4: 한국어 - 에러 조회
📦 Project UUID: 550e8400-e29b-41d4-a716-446655440000
❓ 질문: 최근에 어떤 에러가 발생했어?

✅ 응답 생성 완료 (5.19초)

💡 답변:
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  최근에 다음과 같은 에러가 발생했습니다:

1. **사용자 서비스 (user-service)**에서 두 건의 NullPointerException이 발생했습니다:
   - **Timestamp**: 2025-01-02T10:30:00Z
     - **메서드**: UserService.getUser()
     - **메시지**: NullPointerException - 'user'가 null이어서 'User.getName()'를 호출할 수 없음
   - **Timestamp**: 2025-01-02T10:35:00Z
     - **메서드**: UserService.updateUser()
     - **메시지**: NullPointerException - 'user'가 null이어서 'User.setName()'를 호출할 수 없음

2. **주문 서비스 (order-service)**에서 한 건의 오류가 발생했습니다:
   - **Timestamp**: 2025-01-02T10:40:00Z
     - **메시지**: 데이터베이스 연결 시간 초과 (Database connection timeout)
     - **스택 트레이스**: org.springframework.dao.DataAccessResourceFailureException

이 에러들을 해결하기 위해 사용자 데이터가 null인 경우를 처리하는 로직을 추가하는 것이 필요합니다. 또한, 데이터베이스 연결 문제를 해결하기 위해 서버와 데이터베이스 간의 연결 설정을 점검해야 합니다.
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

💬 테스트 2/4: 영어 - 

## 5. 챗봇 체인 테스트 - 대화 히스토리

이전 대화 맥락을 유지하며 연속적인 질문에 응답하는지 확인합니다.

In [23]:
# 테스트 케이스 4: 챗봇 대화 히스토리 (2개)
chatbot_history_tests = [
    {
        "name": "2턴 대화 - 후속 질문",
        "project_uuid": "550e8400-e29b-41d4-a716-446655440000",
        "context_logs": """Log 1:
- ID: 12345
- Timestamp: 2025-01-02T10:30:00Z
- Service: user-service
- Level: ERROR
- Message: NullPointerException in UserService.getUser()

Log 2:
- ID: 12346
- Timestamp: 2025-01-02T10:35:00Z
- Service: user-service
- Level: ERROR
- Message: NullPointerException in UserService.updateUser()

Log 3:
- ID: 12347
- Timestamp: 2025-01-02T10:40:00Z
- Service: order-service
- Level: ERROR
- Message: Database connection timeout""",
        "history": [
            HumanMessage(content="최근 에러 알려줘"),
            AIMessage(content="최근 발생한 에러는 3건입니다:\n1. UserService의 NullPointerException 2건\n2. OrderService의 Database connection timeout 1건")
        ],
        "question": "그 중에서 가장 심각한 건?"
    },
    {
        "name": "3턴 대화 - 연속 질문",
        "project_uuid": "550e8400-e29b-41d4-a716-446655440000",
        "context_logs": """Log 1:
- ID: 12347
- Timestamp: 2025-01-02T11:00:00Z
- Service: order-service
- Level: ERROR
- Message: Database connection timeout
- Stack Trace: Connection pool exhausted (active: 10/10)""",
        "history": [
            HumanMessage(content="Database connection timeout 에러가 뭐야?"),
            AIMessage(content="Database connection timeout은 데이터베이스 연결 풀이 고갈되어 새로운 연결을 얻을 수 없을 때 발생합니다. 현재 활성 연결이 10/10으로 모두 사용 중입니다."),
            HumanMessage(content="어떻게 해결해?"),
            AIMessage(content="해결 방법:\n1. 연결 풀 크기 증가\n2. 연결 타임아웃 설정 검토\n3. 장기 실행 쿼리 최적화\n4. 연결 누수 확인")
        ],
        "question": "연결 풀 크기는 어떻게 늘려?"
    }
]

print(f"📋 챗봇 대화 히스토리 테스트: {len(chatbot_history_tests)}개 케이스 준비 완료")

📋 챗봇 대화 히스토리 테스트: 2개 케이스 준비 완료


In [24]:
# 챗봇 대화 히스토리 테스트 실행
chatbot_history_results = []

for i, test_case in enumerate(chatbot_history_tests, 1):
    print(f"\n{'='*70}")
    print(f"💬 테스트 {i}/{len(chatbot_history_tests)}: {test_case['name']}")
    print(f"📦 Project UUID: {test_case['project_uuid']}")
    print(f"📜 대화 히스토리: {len(test_case['history'])}턴")
    print(f"❓ 현재 질문: {test_case['question']}")
    print(f"{'='*70}")
    
    # 대화 히스토리 출력
    print(f"\n🔄 대화 맥락:")
    for j, msg in enumerate(test_case['history'], 1):
        role = "👤 사용자" if isinstance(msg, HumanMessage) else "🤖 AI"
        print(f"  {role}: {msg.content[:80]}..." if len(msg.content) > 80 else f"  {role}: {msg.content}")
    
    start_time = time.time()
    
    try:
        # 챗봇 체인 실행 (히스토리 포함, 비동기)
        response = await chatbot_chain.ainvoke({
            "context_logs": test_case['context_logs'],
            "question": test_case['question'],
            "chat_history": test_case['history']
        })
        
        elapsed_time = time.time() - start_time
        answer = response.content
        
        print(f"\n✅ 응답 생성 완료 ({elapsed_time:.2f}초)")
        print(f"\n💡 답변:")
        print(f"  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
        print(f"  {answer}")
        print(f"  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
        
        chatbot_history_results.append({
            "test_name": test_case['name'],
            "project_uuid": test_case['project_uuid'],
            "history_turns": len(test_case['history']),
            "question": test_case['question'][:50],
            "status": "✅ 성공",
            "time": f"{elapsed_time:.2f}s",
            "answer_length": len(answer),
            "answer_preview": answer[:100] + "..." if len(answer) > 100 else answer
        })
        
    except Exception as e:
        elapsed_time = time.time() - start_time
        print(f"\n❌ 실패: {e}")
        chatbot_history_results.append({
            "test_name": test_case['name'],
            "project_uuid": test_case['project_uuid'],
            "history_turns": len(test_case['history']),
            "question": test_case['question'][:50],
            "status": "❌ 실패",
            "time": f"{elapsed_time:.2f}s",
            "error": str(e)[:100]
        })


💬 테스트 1/2: 2턴 대화 - 후속 질문
📦 Project UUID: 550e8400-e29b-41d4-a716-446655440000
📜 대화 히스토리: 2턴
❓ 현재 질문: 그 중에서 가장 심각한 건?

🔄 대화 맥락:
  👤 사용자: 최근 에러 알려줘
  🤖 AI: 최근 발생한 에러는 3건입니다:
1. UserService의 NullPointerException 2건
2. OrderService의 Datab...

✅ 응답 생성 완료 (1.84초)

💡 답변:
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  가장 심각한 에러는 "Database connection timeout"으로, 이는 서비스의 정상적인 운영에 큰 영향을 미칠 수 있습니다. 이 에러는 OrderService에서 발생했으며, 2025년 1월 2일 10:40:00 UTC에 기록되었습니다.
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

💬 테스트 2/2: 3턴 대화 - 연속 질문
📦 Project UUID: 550e8400-e29b-41d4-a716-446655440000
📜 대화 히스토리: 4턴
❓ 현재 질문: 연결 풀 크기는 어떻게 늘려?

🔄 대화 맥락:
  👤 사용자: Database connection timeout 에러가 뭐야?
  🤖 AI: Database connection timeout은 데이터베이스 연결 풀이 고갈되어 새로운 연결을 얻을 수 없을 때 발생합니다. 현재 활성 연결...
  👤 사용자: 어떻게 해결해?
  🤖 AI: 해결 방법:
1. 연결 풀 크기 증가
2. 연결 타임아웃 설정 검토
3. 장기 실행 쿼리 최적화
4. 연결 누수 확인

✅ 응답 생성 완료 (4.21초)

💡 답변:
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  연결 풀 크기를 늘리려면 다음 단계를 따르세요:

1. **설정 파일 열기**: 데이터베이스 연결 설정이 포함된 파일을 찾아

## 6. 전체 결과 요약

In [25]:
import pandas as pd

print("\n" + "="*70)
print("🎯 전체 테스트 결과 요약")
print("="*70)

# 통계 계산
all_results = [
    *single_log_results,
    *trace_log_results,
    *chatbot_basic_results,
    *chatbot_history_results
]

total_tests = len(all_results)
total_success = sum(1 for r in all_results if '✅' in r['status'])
total_failed = total_tests - total_success

print(f"\n📊 전체 통계:")
print(f"  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
print(f"  총 테스트: {total_tests}개")
print(f"  ✅ 성공: {total_success}개")
print(f"  ❌ 실패: {total_failed}개")
print(f"  📈 성공률: {total_success/total_tests*100:.1f}%")

# 카테고리별 통계
print(f"\n📈 카테고리별 결과:")
print(f"  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")

categories = [
    ("단일 로그 분석", single_log_results),
    ("Trace 로그 분석", trace_log_results),
    ("챗봇 기본 질의", chatbot_basic_results),
    ("챗봇 대화 히스토리", chatbot_history_results)
]

for category_name, results in categories:
    if results:
        success = sum(1 for r in results if '✅' in r['status'])
        total = len(results)
        rate = (success/total*100) if total > 0 else 0
        print(f"  {category_name}: {success}/{total} ({rate:.1f}%)")

# 최종 결과
print(f"\n{'='*70}")
if total_success == total_tests:
    print("✅✅✅ 모든 테스트 통과! 🎉🎉🎉")
elif total_success >= total_tests * 0.8:
    print(f"⚠️ {total_failed}개 테스트 실패 (대부분 통과)")
else:
    print(f"❌ {total_failed}개 테스트 실패 (개선 필요)")
print("="*70)


🎯 전체 테스트 결과 요약

📊 전체 통계:
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  총 테스트: 11개
  ✅ 성공: 11개
  ❌ 실패: 0개
  📈 성공률: 100.0%

📈 카테고리별 결과:
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  단일 로그 분석: 3/3 (100.0%)
  Trace 로그 분석: 2/2 (100.0%)
  챗봇 기본 질의: 4/4 (100.0%)
  챗봇 대화 히스토리: 2/2 (100.0%)

✅✅✅ 모든 테스트 통과! 🎉🎉🎉


In [26]:
# 결과 테이블 출력
print("\n📋 상세 결과 테이블:\n")

if single_log_results:
    print("\n1️⃣ 단일 로그 분석:")
    df1 = pd.DataFrame(single_log_results)
    display(df1)

if trace_log_results:
    print("\n2️⃣ Trace 로그 분석:")
    df2 = pd.DataFrame(trace_log_results)
    display(df2)

if chatbot_basic_results:
    print("\n3️⃣ 챗봇 기본 질의:")
    df3 = pd.DataFrame(chatbot_basic_results)
    display(df3)

if chatbot_history_results:
    print("\n4️⃣ 챗봇 대화 히스토리:")
    df4 = pd.DataFrame(chatbot_history_results)
    display(df4)


📋 상세 결과 테이블:


1️⃣ 단일 로그 분석:


,test_name,project_uuid,status,time,summary,tags_count,has_error_cause,has_solution
0,ERROR - NullPointerException,550e8400-e29b-41d4-a716-446655440000,✅ 성공,2.71s,NullPointerException in UserService.getUser() ...,3,Yes,Yes
1,ERROR - Database Connection Timeout,550e8400-e29b-41d4-a716-446655440000,✅ 성공,2.82s,Database connection timeout in OrderRepository...,4,Yes,Yes
2,INFO - 정상 처리,550e8400-e29b-41d4-a716-446655440000,✅ 성공,2.21s,Payment processed successfully in PaymentService.,3,No,No



2️⃣ Trace 로그 분석:


,test_name,project_uuid,trace_id,total_logs,status,time,summary,tags_count
0,Trace - 5개 관련 로그 (API → Service → Error),550e8400-e29b-41d4-a716-446655440000,abc123def456,5,✅ 성공,2.78s,NullPointerException in UserService.getUserByI...,5
1,Trace - 12개 로그 (Map-Reduce 트리거),550e8400-e29b-41d4-a716-446655440000,xyz789ghi012,12,✅ 성공,3.12s,Order creation process failed due to a payment...,4



3️⃣ 챗봇 기본 질의:


,test_name,project_uuid,question,status,time,answer_length,answer_preview
0,한국어 - 에러 조회,550e8400-e29b-41d4-a716-446655440000,최근에 어떤 에러가 발생했어?,✅ 성공,5.19s,763,최근에 다음과 같은 에러가 발생했습니다:\n\n1. **사용자 서비스 (user-s...
1,영어 - 에러 조회,550e8400-e29b-41d4-a716-446655440000,What errors occurred recently?,✅ 성공,5.13s,586,최근 발생한 오류는 다음과 같습니다:\n\n1. **주문 서비스 (order-ser...
2,한국어 - 해결 방법 질문,550e8400-e29b-41d4-a716-446655440000,NullPointerException을 어떻게 해결하면 돼?,✅ 성공,5.51s,641,NullPointerException을 해결하기 위해서는 다음과 같은 단계들을 고려...
3,컨텍스트 없음 - 일반 질문,550e8400-e29b-41d4-a716-446655440000,최근 에러 알려줘,✅ 성공,0.95s,53,최근 로그에서 관련된 에러 정보는 찾을 수 없습니다. 추가적인 도움이 필요하시면 말...



4️⃣ 챗봇 대화 히스토리:


,test_name,project_uuid,history_turns,question,status,time,answer_length,answer_preview
0,2턴 대화 - 후속 질문,550e8400-e29b-41d4-a716-446655440000,2,그 중에서 가장 심각한 건?,✅ 성공,1.84s,140,"가장 심각한 에러는 ""Database connection timeout""으로, 이는..."
1,3턴 대화 - 연속 질문,550e8400-e29b-41d4-a716-446655440000,4,연결 풀 크기는 어떻게 늘려?,✅ 성공,4.21s,477,연결 풀 크기를 늘리려면 다음 단계를 따르세요:\n\n1. **설정 파일 열기**:...


## 7. 응답 시간 분석

In [27]:
# 응답 시간 통계
def extract_time(time_str):
    try:
        return float(time_str.replace('s', ''))
    except:
        return None

successful_results = [r for r in all_results if '✅' in r['status']]

if successful_results:
    times = [extract_time(r['time']) for r in successful_results if extract_time(r['time']) is not None]
    
    if times:
        print(f"\n⏱️ 응답 시간 통계:")
        print(f"  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
        print(f"  평균: {sum(times)/len(times):.2f}초")
        print(f"  최소: {min(times):.2f}초")
        print(f"  최대: {max(times):.2f}초")
        print(f"  중간값: {sorted(times)[len(times)//2]:.2f}초")
else:
    print("\n⚠️ 성공한 테스트가 없어 응답 시간을 분석할 수 없습니다.")


⏱️ 응답 시간 통계:
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  평균: 3.32초
  최소: 0.95초
  최대: 5.51초
  중간값: 2.82초


## 8. 결론

**테스트 완료!**

이 노트북은 다음을 확인했습니다:
- ✅ LangChain 체인이 올바르게 로드되는지
- ✅ 단일 로그 분석이 정확하게 작동하는지
- ✅ Trace 기반 다중 로그 분석이 정확하게 작동하는지
- ✅ 챗봇이 한국어/영어 질문에 적절히 응답하는지
- ✅ 챗봇이 대화 히스토리를 유지하며 응답하는지
- ✅ project_uuid를 사용한 multi-tenancy 지원
- ✅ LLM 응답 품질 및 응답 시간

**다음 단계**:
1. 실패한 테스트가 있다면 원인 분석 및 수정
2. 전체 시스템 통합 테스트 (FastAPI 서버, OpenSearch 포함)
3. 프로덕션 배포 전 성능 테스트 및 부하 테스트